# Simple Disease CSV — Analysis & Model Building
Is notebook me hum dataset.csv load karenge, symptoms ko combine karenge, ek chhota RandomForest model train karenge aur artifacts save karenge.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

BASE_DIR = Path('.').resolve()
DATA_PATH = BASE_DIR / 'dataset.csv'   # dataset.csv ko notebook folder me rakho
ARTIFACTS_DIR = BASE_DIR / 'artifacts'
ARTIFACTS_DIR.mkdir(exist_ok=True)
print("DATA_PATH =", DATA_PATH)

In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"dataset.csv nahi mila: {DATA_PATH}")

    df = pd.read_csv(DATA_PATH)
    df.columns = [c.strip().replace(' ', '_') for c in df.columns]
    print("Shape:", df.shape)
    display(df.head())

In [ ]:
# CSV me symptom wale columns ka naam symptom_1, symptom_2 ... jo bhi hai uske hisab se detect karega
symptom_cols = [c for c in df.columns if 'symptom' in c.lower()]
print("Found symptom cols:", symptom_cols)

# agar mil gaye to join kar do, warna ek error bathao
if not symptom_cols:
    raise RuntimeError("Koi symptom_* column nahi mila. CSV ke columns check karo.")

    df_sym = df[symptom_cols].fillna('').astype(str)
    df['symptom_text'] = df_sym.apply(lambda row: ' '.join(row.str.strip().str.lower()), axis=1)
    display(df[['symptom_text']].head())

In [ ]:
# Target: 'Disease' column use karo
if 'Disease' not in df.columns:
    raise RuntimeError("CSV me 'Disease' column hona chahiye.")

    # Rare labels ko 'Other' me group karna (optional, simple rule)
    min_samples = 2
    vc = df['Disease'].value_counts()
    rare = vc[vc < min_samples].index.tolist()
    df['Disease_clean'] = df['Disease'].apply(lambda x: x if x not in rare else 'Other')

    # TF-IDF
    vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
    X = vectorizer.fit_transform(df['symptom_text'].fillna(''))

    le = LabelEncoder()
    y = le.fit_transform(df['Disease_clean'])

    print("X shape:", X.shape)
    print("Classes:", list(le.classes_))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Test accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("Classification report:")
print(classification_report(y_test, y_pred, target_names=le.inverse_transform(sorted(set(y_test)))))

In [ ]:
joblib.dump(model, ARTIFACTS_DIR / 'disease_model.pkl', compress=('gzip',3))
joblib.dump(vectorizer, ARTIFACTS_DIR / 'tfidf_vectorizer.pkl', compress=('gzip',3))
joblib.dump(le, ARTIFACTS_DIR / 'label_encoder.pkl', compress=('gzip',3))
print("Saved artifacts to", ARTIFACTS_DIR)

In [ ]:
# Sample symptoms (aap change kar sakte ho)
sample = ["fever", "cough", "sore throat"]
input_text = " ".join([s.lower() for s in sample])
X_in = vectorizer.transform([input_text])
pred_idx = model.predict(X_in)[0]
pred_label = le.inverse_transform([pred_idx])[0]
print("Input:", sample)
print("Predicted disease:", pred_label)

# Agar model.predict_proba chahiye:
if hasattr(model, "predict_proba"):
    probs = model.predict_proba(X_in)[0]
        print("Confidence (predicted class prob):", round(probs[pred_idx], 3))